<a href="https://colab.research.google.com/github/ChanseoKim00/finbert-project/blob/main/finbert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
!ls /content/drive/MyDrive


 1079_ec797a6c0b-1.hwp
 1079_fdeccb4d55.hwp
 1138_52f03ec7a3.hwp
 Classroom
'Colab Notebooks'
 FinBERT_Project
'https:  truthman.co.kr -Performance-on-Search-2023-09-05.gsheet'
'IMG_1392 (1).jpeg'
 IMG_1392.jpeg
'제목 없는 스프레드시트.gsheet'


In [12]:
import os

base_path = '/content/drive/MyDrive/FinBERT_Project'
os.makedirs(base_path, exist_ok=True)

print("작업 폴더 생성 완료:", base_path)


작업 폴더 생성 완료: /content/drive/MyDrive/FinBERT_Project


In [13]:
!pip install -q transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# 모델 불러오기
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [14]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze().numpy()

    labels = ["negative", "neutral", "positive"]
    result = {labels[i]: float(scores[i]) for i in range(3)}
    return sorted(result.items(), key=lambda x: -x[1])

# 예제 문장
sentence = "Nvidia stock surges after AI chip demand beats expectations"
result = analyze_sentiment(sentence)
print("Sentiment Analysis Result:")
for label, score in result:
    print(f"{label.capitalize()}: {score:.2f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentiment Analysis Result:
Neutral: 1.00
Positive: 0.00
Negative: 0.00


In [15]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze().numpy()

    labels = ["negative", "neutral", "positive"]
    result = {labels[i]: float(scores[i]) for i in range(3)}
    return sorted(result.items(), key=lambda x: -x[1])

# 예제 문장
sentence = "Nvidia stock surges after AI chip demand beats expectations"
result = analyze_sentiment(sentence)
print("Sentiment Analysis Result:")
for label, score in result:
    print(f"{label.capitalize()}: {score:.2f}")


Sentiment Analysis Result:
Neutral: 1.00
Positive: 0.00
Negative: 0.00


In [16]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.to(torch.float32)  # <-- 여기를 추가!
        scores = torch.nn.functional.softmax(logits, dim=1).squeeze().numpy()

    labels = ["negative", "neutral", "positive"]
    result = {labels[i]: float(scores[i]) for i in range(3)}
    return sorted(result.items(), key=lambda x: -x[1])


In [17]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze().numpy()

    labels = ["negative", "neutral", "positive"]
    result = {labels[i]: float(scores[i]) for i in range(3)}
    return sorted(result.items(), key=lambda x: -x[1])

# 예제 문장
sentence = "Nvidia stock surges after AI chip demand beats expectations"
result = analyze_sentiment(sentence)
print("Sentiment Analysis Result:")
for label, score in result:
    print(f"{label.capitalize()}: {score:.2f}")


Sentiment Analysis Result:
Neutral: 1.00
Positive: 0.00
Negative: 0.00


In [18]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze().numpy()

    labels = ["negative", "neutral", "positive"]
    result = {labels[i]: float(scores[i]) for i in range(3)}
    return sorted(result.items(), key=lambda x: -x[1])

# 예제 문장
sentence = "Nvidia stock rises 12% after soften regulations"
result = analyze_sentiment(sentence)
print("Sentiment Analysis Result:")
for label, score in result:
    print(f"{label.capitalize()}: {score:.2f}")


Sentiment Analysis Result:
Neutral: 1.00
Negative: 0.00
Positive: 0.00


In [20]:
!pip install feedparser


In [21]:
import feedparser
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# 모델 및 토크나이저 로드
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits.to(torch.float32)
        scores = torch.nn.functional.softmax(logits, dim=1).squeeze().numpy()
    labels = ["negative", "neutral", "positive"]
    return {labels[i]: float(scores[i]) for i in range(3)}

def get_google_news_titles(keyword, max_items=10):
    feed_url = f"https://news.google.com/rss/search?q={keyword}+stock&hl=en-US&gl=US&ceid=US:en"
    feed = feedparser.parse(feed_url)
    return [entry.title for entry in feed.entries[:max_items]]

# 키워드 입력
keyword = "Nvidia"
titles = get_google_news_titles(keyword)

# 결과 분석
results = []
for title in titles:
    scores = analyze_sentiment(title)
    results.append({
        "title": title,
        "positive": round(scores["positive"], 3),
        "neutral": round(scores["neutral"], 3),
        "negative": round(scores["negative"], 3)
    })

# 표 형태로 출력
df = pd.DataFrame(results)
df.sort_values("positive", ascending=False).reset_index(drop=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,title,positive,neutral,negative
0,Nvidia Stock Drops as Trump Trade Fears Hit th...,0.990,0.004,0.005
1,Super Micro Stock (SMCI) Jumps About 60% — Is ...,0.001,0.819,0.180
2,Nvidia-backed stock sends a quiet shockwave th...,0.000,0.000,1.000
3,Where Will Nvidia Stock Be in 1 Year? - Yahoo ...,0.000,0.000,1.000
4,Nvidia Stock Is on a Tear. How It Can Beat App...,0.000,1.000,0.000
5,Better Artificial Intelligence Stock: Nvidia v...,0.000,1.000,0.000
6,"Tesla, Nvidia, Shell, Netflix, and More Stock ...",0.000,0.493,0.507
7,Prediction: Nvidia Will Soar in the Second Hal...,0.000,0.903,0.097
8,Nvidia stock is on a tear. This ‘next big thin...,0.000,0.999,0.001
9,Nvidia's stock gains put it nearly on track to...,0.000,1.000,0.000
